# CBCT压缩、分割与融合

！该接口处于BETA状态！

In [ ]:
# 导入必要的包以及定义函数
import os
import glob
import base64
import time
import requests
import json
import trimesh
import urllib
import numpy as np

def _create_colors():
    # 20 high contrast colors
    colors = [[230, 25, 75,255],[60, 180, 75,255],[255, 225, 25,255],\
            [0, 130, 200,255],[245, 130, 48,255],[145, 30, 180,255],[70, 240, 240,255],\
            [240, 50, 230,255],[210, 245, 60,255],[250, 190, 190,255],[0, 128, 128,255],\
            [230, 190, 255,255],[170, 110, 40,255],[255, 250, 200,255],[128, 0, 0,255],\
            [170, 255, 195,255],[128, 128, 0, 255]]
    #np.random.shuffle(colors)
    # gum color
    colors = [[255,255,255,255]] + colors
    return colors

def colored_mesh(mesh, cid):
    COLORS = _create_colors()
    mcopy = mesh.copy()
    mcopy.visual.face_colors = COLORS[cid % 18]
    return mcopy

# 定义调用规则

请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # 必须指定 postfix, 即文件后缀名
                        headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

    resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

def run_job_and_get_results(json_call, timeout_sec):
    headers = {
      "Content-Type": "application/json",
      "X-ZH-TOKEN": zh_token
    }

    url = base_url + '/run'

    response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
    response.raise_for_status()
    create_result = response.json()
    run_id = create_result['run_id']
    print("workflow id is", run_id)
    url = base_url + f"/run/{run_id}"

    start_time = time.time()
    while time.time()-start_time < timeout_sec:
        time.sleep(0.3)
        response = requests.request("GET", url, headers=headers)
        result = response.json()
        if result['completed'] or result['failed']:
            break

    if not result['completed']:
        if result['failed']:
            raise ValueError("API failed due to " + str(result['reason_public']))
        raise TimeoutError("API timeout")

    print("API finished in {}s".format(time.time()-start_time))
    url = base_url + f"/data/{run_id}"
    response = requests.request("GET", url, headers=headers)
    return response.json()

def retrieve_data(urn):
    return requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": urn}),
                        headers={"X-ZH-TOKEN": zh_token}).content

def retrieve_mesh(mesh_file_json):
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": mesh_file_json['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    return trimesh.load(trimesh.util.wrap_as_stream(resp.content), file_type=mesh_file_json['type'])

## CBCT压缩

CBCT压缩算法可以进行有损CBCT压缩，生成的CBCT用于可视化或分析（分辨率不变）。接口的压缩率为50～100倍（430MB的NRRD文件压缩后为8.4M）

具体接口见 https://www.chohotech.com/docs/cloud-zh/#/module/cbct-compression-1

In [ ]:
json_call = {
  "spec_group": "cbct", # 调用的工作流组， 随API文档发送
  "spec_name": "cbct-compression", # 调用的工作流名称， 随API文档发送, 
  "spec_version": "1.0-snapshot", # 调用的工作流版本，随API文档发送
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"ct_file": upload_file("cbct.zip")}
}
result = run_job_and_get_results(json_call, 300)

In [ ]:
print("compressed data is " + str(len(retrieve_data(result['result']['zhct0'])) / 1024 / 1024) + "MB")

## 根骨重建

In [ ]:
json_call = {
  "spec_group": "cbct",
  "spec_name": "cbct-reconstruction",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {"zhct0": result['result']['zhct0']}, # You can directly use urn here without downloading / uploading the file again!
  "output_config": {
      "meshes": {"type": "ply"},
      "lower_teeth": {"type": "ply"},
      "upper_teeth": {"type": "ply"}
  }
}
result = run_job_and_get_results(json_call, 300)

In [ ]:
meshes = sum([
    colored_mesh(retrieve_mesh(result['result']["meshes"][k]), i+2) for i,k in enumerate(result['result']["meshes"].keys())
], None)

In [ ]:
meshes.show()

## 根冠融合

根冠融合需按半颌调用，此处仅以上颌为例。具体接口可参考 https://www.chohotech.com/docs/cloud-zh/#/module/cbct-scan-fusion-1

In [ ]:
# store results from reconstruction
cbct_upper_teeth = result['result']['upper_teeth']

# run teeth segmentation to get oral scan teeth
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-comp-and-axis",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
     "mesh": {"type":"drc", "data": upload_file('upper_jaw_scan.drc')},
     "jaw_type": "Upper"
  }
}
result = run_job_and_get_results(json_call, 300)

In [ ]:
# run teeth segmentation to get oral scan teeth
json_call = {
  "spec_group": "cbct",
  "spec_name": "cbct-scan-fusion",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
     "cbct_mesh": cbct_upper_teeth,
     "teeth": result['teeth_comp']
  },
  "output_config": {
      "teeth": {"type": "ply"},
      "scan_teeth": {"type": "ply"},
      "fusion_teeth": {"type": "ply"}
  }
}
result = run_job_and_get_results(json_call, 300)

In [ ]:
meshes = sum([
    colored_mesh(retrieve_mesh(result['result']["fusion_teeth"][k]), i+2) for i,k in enumerate(result['result']["fusion_teeth"].keys())
], None)

In [ ]:
meshes.show()